In [1]:
from snowflake.nodes import *
from snowflake.stencil_compiler import *
from snowflake.analytics import get_shadow, validate_stencil
from snowflake.utils import swap_variables
import numpy as np
import inspect
!ctree -cc

ctree looking for relative cache directories named compiled, checking directories under this one



In [2]:
data = np.arange(64, dtype=np.float64)
swap = np.zeros_like(data)

In [3]:
smooth_sc = StencilComponent('data', WeightArray([1/4.0, 1/2.0, 1/4.0]))
smooth_stencil = Stencil(smooth_sc, 'swap', ((1, -1, 1),))

In [4]:
boundary_left_sc = StencilComponent('swap', WeightArray([0, 0, -1]))
boundary_left_stencil = Stencil(boundary_left_sc, 'swap', ((0, 1, 1),))

In [5]:
boundary_right_sc = StencilComponent('swap', WeightArray([-1, 0, 0]))
boundary_right_stencil = Stencil(boundary_right_sc, 'swap', ((-1, 0, 1),))

In [6]:
compiler = CCompiler()

In [7]:
sk = compiler.compile(smooth_stencil)
blk = compiler.compile(boundary_left_stencil)
brk = compiler.compile(boundary_right_stencil)

In [8]:
sk(swap, data)
print(swap)
blk(swap)
print(swap)
brk(swap)
print(swap)

[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.
  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.  43.  44.
  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.  57.  58.  59.
  60.  61.  62.   0.]
[ -1.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.
  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.  43.  44.
  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.  57.  58.  59.
  60.  61.  62.   0.]
[ -1.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.
  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.  43.  44.
  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.  57.  58.  59.
  60.  61.  62. -62.]


In [9]:
swap

array([ -1.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62., -62.])

In [10]:
sk(data, swap)

In [11]:
data

array([  0.  ,   0.75,   2.  ,   3.  ,   4.  ,   5.  ,   6.  ,   7.  ,
         8.  ,   9.  ,  10.  ,  11.  ,  12.  ,  13.  ,  14.  ,  15.  ,
        16.  ,  17.  ,  18.  ,  19.  ,  20.  ,  21.  ,  22.  ,  23.  ,
        24.  ,  25.  ,  26.  ,  27.  ,  28.  ,  29.  ,  30.  ,  31.  ,
        32.  ,  33.  ,  34.  ,  35.  ,  36.  ,  37.  ,  38.  ,  39.  ,
        40.  ,  41.  ,  42.  ,  43.  ,  44.  ,  45.  ,  46.  ,  47.  ,
        48.  ,  49.  ,  50.  ,  51.  ,  52.  ,  53.  ,  54.  ,  55.  ,
        56.  ,  57.  ,  58.  ,  59.  ,  60.  ,  61.  ,  30.75,  63.  ])

In [12]:
brk(data)
blk(data)

In [13]:
data

array([ -0.75,   0.75,   2.  ,   3.  ,   4.  ,   5.  ,   6.  ,   7.  ,
         8.  ,   9.  ,  10.  ,  11.  ,  12.  ,  13.  ,  14.  ,  15.  ,
        16.  ,  17.  ,  18.  ,  19.  ,  20.  ,  21.  ,  22.  ,  23.  ,
        24.  ,  25.  ,  26.  ,  27.  ,  28.  ,  29.  ,  30.  ,  31.  ,
        32.  ,  33.  ,  34.  ,  35.  ,  36.  ,  37.  ,  38.  ,  39.  ,
        40.  ,  41.  ,  42.  ,  43.  ,  44.  ,  45.  ,  46.  ,  47.  ,
        48.  ,  49.  ,  50.  ,  51.  ,  52.  ,  53.  ,  54.  ,  55.  ,
        56.  ,  57.  ,  58.  ,  59.  ,  60.  ,  61.  ,  30.75, -30.75])

In [14]:
def smooth1(data, i):
    tmp = np.zeros_like(data)
    for k in range(i):
        sk(tmp, data)
        brk(tmp)
        blk(tmp)
        tmp, data = data, tmp
    return data

In [15]:
data = np.ones(64, dtype=np.float64)

In [16]:
# sum(smooth1(data, 10000))

In [17]:
data = np.ones(64, dtype=np.float64)

In [18]:
combined = StencilGroup([smooth_stencil, boundary_right_stencil, boundary_left_stencil])
print(combined.body)

[<snowflake.nodes.Stencil object at 0x10d8b4c10>, <snowflake.nodes.Stencil object at 0x10d8b4ed0>, <snowflake.nodes.Stencil object at 0x10d8b4c90>]


In [19]:
ck = compiler.compile(combined)
print(ck.arg_spec)

['swap', 'data']


In [20]:
def smooth2(data, i):
    tmp = np.zeros_like(data)
    for k in range(i):
        ck(tmp, data)
        data, tmp = tmp, data
    return data

In [21]:
data

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [22]:
data = np.arange(100)
out = np.zeros_like(data)
for i in range(5):
    ck(out, data)
    print(out)
    out, data = data, out

[ -1   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98 -98]
[  0   0   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  48 -48]
[  0   0   1   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45

In [23]:
out

array([  0,   0,   1,   2,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  92,  75,  30, -30])

In [24]:
bool(boundary_right_sc[(-1,)])

False

In [25]:
get_shadow(smooth_stencil)

defaultdict(set, {'data': {(-1), (0), (1)}})

In [26]:
validate_stencil(smooth_stencil)

True

In [31]:
print(dump(boundary_right_sc))

StencilComponent(name='swap', weights=WeightArray(weights=[
    StencilConstant(value=-1),
    StencilConstant(value=0),
    StencilConstant(value=0),
  ]))


In [32]:
swapped = swap_variables(boundary_right_sc, {'swap':'data', 'data':'swap'})
print(dump(swapped))

StencilComponent(name='data', weights=WeightArray(weights=[
    StencilConstant(value=-1),
    StencilConstant(value=0),
    StencilConstant(value=0),
  ]))


In [33]:
print(dump(swap_variables(swapped, {'swap':'data', 'data':'swap'})))

StencilComponent(name='swap', weights=WeightArray(weights=[
    StencilConstant(value=-1),
    StencilConstant(value=0),
    StencilConstant(value=0),
  ]))
